In [283]:
import requests
import re
import json
from pyproj import Proj, transform
import twd97
import geocoder
# address="桃園市中壢區永泰街110號1F"
# address="台中市西區台灣大道二段218號"
# address="台北市松山區八德路二段312號"
# address="雲林縣斗六市雲林路二段297號"
# address="台中市西區民權路"

address=input("請輸入地址:")

bigadd =""
smalladd=""

try:
    bigadd = re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",address)[0][0]
    smalladd= re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",address)[0][1]
except:
    pass
try:
    road=address.split(smalladd)[1]
except:
    road=address
    print("沒有市或區，可能會出問題")

headersStr="""Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 114
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/WEB/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest"""
headerDict={i.split(":")[0]:i.split(":")[1].strip() for i in headersStr.split("\n")}

dataDict={}
dataDict["VAL1"]=road
if bigadd!="":
    dataDict["COUNTY"]=bigadd
    dataDict["TOWN"]=smalladd
    findXYUrl="http://luz.tcd.gov.tw/WEB/ws_data.ashx?CMD=GETADDRESS"
    res=requests.post(findXYUrl,data=dataDict,headers=headerDict)
    locationData=json.loads(res.text)
    lat,lng=twd97.towgs84(locationData['AddressList'][0]['X'],locationData['AddressList'][0]['Y'])
else:
    lat,lng=geocoder.google(address).latlng
    print(lat,lng)

#以上由地址轉換出TWD97的座標

#由TWD97的座標轉換成經緯度



#由經緯度轉換成epsg3857座標
P4326 = Proj(init='epsg:4326')
P3857 = Proj(init='epsg:3857')
x,y = transform(P4326, P3857, lng, lat)


queryHeadersStr="""Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 69
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/WEB/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest"""

queryHeaderDict={i.split(":")[0]:i.split(":")[1].strip() for i in queryHeadersStr.split("\n")}

queryData={}
queryData["X"],queryData["Y"]=x,y
# queryData["X"],queryData["Y"]=13495533.107782178,2870575.0923182988

queryData["LAYERS"]="5,3,6"
queryData['LANDLAYERS']=""
queryUrl="http://luz.tcd.gov.tw/WEB/ws_identify.ashx"
res=requests.post(queryUrl,data=queryData,headers=queryHeaderDict)
info=json.loads(res.text)

請輸入地址:台中市西區台灣大道二段218-49號


In [284]:
info

{'3': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}},
 '5': {'features': [{'attributes': {'OBJECTID': '145705',
     '使用分區': '第三種住宅區',
     '備註': '',
     '分區簡稱': '住',
     '參考面積': '1816.21255800926',
     '計畫區名稱': '臺中市都市計畫(不包括大坑風景區)',
     '計畫案名稱': '變更臺中市都市計畫(舊有市區及一至五期市地重劃地區)細部計畫(第二次通盤檢討)案',
     '變更前分區': '',
     '資料品質': '',
     '資料日期': '',
     '附帶條件': ''}}],
  'geometryType': '',
  'spatialReference': {'wkid': 3857}},
 '6': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}}}

In [88]:
XX_train=[[304933.651,2771157.358],
          [273471.540,2761142.866],
          [216372.891,2671908.212],
          [216412.591,2671855.832],
          [216423.471,2671841.271],
          [273610.496,2761109.589],
          [273586.274,2761163.312],
          [273456.939,2761136.920],
          [273395.027, 2761141.153],
          [216485.151, 2670764.174],
          [304633.65, 2770157.35],
          [314633.65, 2970157.35],
          [300933.651,2700157.358]
         ]

YY_train=[[13530262.995185157, 2881588.428958615],
          [13495534.149093753,2870579.1198068312],
          [13432821.756902112,2771949.264442703],
          [13432865.380201073,2771891.3695835327],
          [13432877.037296467,2771871.1779812025],
          [13495687.1564441,2870542.009716954],
          [13495660.55498392,2870601.312034573],
          [13495517.964723885,2870572.3010908756],
          [13495449.673774207, 2870577.409433772],
          [13432947.709334468, 2770689.3807398593],
          [13529927.787628435, 2880480.6334838117],
          [13542054.133565776, 3103930.6888663652],
          [13525563.22124938, 2803044.487542941]
         ]


In [89]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(XX_train, YY_train)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [177]:
lm.predict([[304633.65,2770157.35]])

array([[13529834.92704136,  2880554.22952524]])

In [91]:
xxx='''"X": 304633.65,
      "Y": 2770157.35'''

In [92]:
[float(i.split(":")[1].strip()) for i in xxx.split(',\n')]

[304633.65, 2770157.35]

In [93]:
yyy='''X: 13525563.22124938
Y: 2803044.487542941'''

In [94]:
[float(i.split(":")[1].strip()) for i in yyy.split('\n')]

[13525563.22124938, 2803044.487542941]

In [95]:
from pyproj import Proj, transform

In [96]:
import geocoder

In [135]:
lat,lng=geocoder.arcgis("台中市西區台灣大道二段218號").latlng

In [156]:
P4326 = Proj(init='epsg:4326')

In [157]:
P3857 = Proj(init='epsg:3857')

In [158]:
x,y = transform(P4326, P3857, lng, lat)

In [159]:
x,y

(13525563.520960934, 2803045.0939111193)

In [178]:
import requests
import json
queryHeadersStr="""Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 68
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/web/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest"""

queryHeaderDict={i.split(":")[0]:i.split(":")[1].strip() for i in queryHeadersStr.split("\n")}

queryData={}
queryData["X"],queryData["Y"]=x,y#TWD97
# queryData["X"],queryData["Y"]=13495533.107782178,2870575.0923182988

queryData["LAYERS"]="5,3,6"
queryData['LANDLAYERS']=""
queryUrl="http://luz.tcd.gov.tw/WEB/ws_identify.ashx"
res=requests.post(queryUrl,data=queryData,headers=queryHeaderDict)
info=json.loads(res.text)

In [179]:
info

{'3': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}},
 '5': {'features': [{'attributes': {'OBJECTID': '57627',
     '使用分區': '軍事機關用地',
     '備註': '',
     '分區簡稱': '軍事機關',
     '參考面積': '71972.16',
     '計畫區名稱': '臺北市都市計畫',
     '計畫案名稱': '106年現行計畫圖',
     '變更前分區': '',
     '資料品質': '',
     '資料日期': '',
     '附帶條件': ''}}],
  'geometryType': '',
  'spatialReference': {'wkid': 3857}},
 '6': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}}}

In [147]:
import twd97

In [155]:
lat,lng=twd97.towgs84(300933.651,2700157.358)